In [1]:
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd
import csv
import datetime
import matplotlib.pyplot as plt
from datapathfile import path_lookup_dir, datafile

In [2]:
def listcolumn(x):
    x = x.translate(None, '"')
    x = x.translate(None, ' ')
    x = x[1:-1].split(",")
    j = x[-1]
    return j

In [3]:
def TrueFalse(value):
    if value[0]=='F':
        value = False
    else:
        value = True
    return value

In [4]:
reader = list(csv.reader(open(path_lookup_dir+datafile, 'rb'), delimiter='\t'))
colheaders = reader[0] 
df = pd.DataFrame(reader[1:])
dffilter = df.iloc[:, 0:20]
dffilter.columns = [colheaders]

In [5]:
dffilter['installed'].unique()

array(['False', 'False2015-05-16 00:01:18', 'False2015-05-17 06:47:28',
       ..., 'False2015-07-24 16:45:05', 'False2015-07-24 16:51:24',
       'False2015-07-24 17:20:35'], dtype=object)

In [6]:
df = dffilter

In [7]:
df['impression'] = df['impression'].apply(TrueFalse)

In [8]:
df['clicked'] = df['clicked'].apply(TrueFalse)

In [9]:
df.ix[:, 'datetime'] = pd.to_datetime(df['datetime'], format="%Y-%m-%d %H:%M:%S")
df.ix[:, 'app_type'] = df['app_categories'].apply(listcolumn)


In [10]:
drop_index=df[np.logical_and(
        np.logical_and(df['impression'], df['installed']),
        ~ df['clicked'])].index
empty_index = np.where(df['app_type'] == '')
ul_index = np.where(df['app_type'] == 'ul')
df = df.drop(df.index[empty_index])
df = df.drop(df.index[ul_index])
df = df.drop(df.index[drop_index])

In [11]:
df['weekday'] = df['datetime'].map(lambda x: x.isoweekday())
df['hours'] = df['datetime'].map(lambda x: x.hour)
df.ix[:, 'week'] = df['datetime'].map(lambda x:x.isocalendar()[1])

In [12]:
impressiondf = df.groupby('city')['impression','clicked'].sum()
impression_index = impressiondf[impressiondf['impression'] > 5].index

In [13]:
df=df[df.city.isin(impression_index)]

In [14]:
filename='ATA_data.csv'

In [15]:
df.to_csv(filename)